In [ ]:
import pandas as pd
import glob
import itertools

# (경우1) 월 결과(월 MSE)의 target명 통일시키기

In [ ]:
## 월 결과(월 MSE)의 target명 통일시키기
file_list = glob.glob('result/*.csv')
file_list.sort()

for file_nm in file_list:
    tmp_df = pd.read_csv(file_nm, encoding = 'utf-8')
    tmp_df = tmp_df.drop_duplicates(['target'], keep='first')
    tmp_df.reset_index(drop=True, inplace=True)
#     tmp_df = tmp_df[['target','MSE','BEST_MDL']]
    if file_nm.find('dis') == -1:
        tmp_df['target1'] = tmp_df['target'].map(lambda x : x.split('_')[0])
        tmp_df['target2'] = tmp_df['target'].map(lambda x : x.split('_')[1])
    else:
        tmp_df['target1'] = tmp_df['target'].map(lambda x : x.split('_')[0])
        tmp_df['target2'] = tmp_df['target'].map(lambda x : x.split('_')[1])        
        tmp_df['target3'] = tmp_df['target'].map(lambda x : x.split('_')[2])
        
    # < target 컬럼명 조정 >
    # 1-1. 기본 data의 sido 컬럼 호출
    data_list = glob.glob('data/*.csv')
    sido = []
    for data_nm in data_list:
        tmp = pd.read_csv(data_nm, encoding = 'cp949')
        sido.append(list(tmp['SIDO'].unique()))

    sido = list(pd.Series(list(itertools.chain.from_iterable(sido))).unique())
    sido.sort()
    
    # 1-2. result 데이터의 sido 컬럼 호출
    target1 = list(tmp_df['target1'].unique())
    target1.sort()
    target2 = list(tmp_df['target2'].unique())
    target2.sort()
    if file_nm.find('dis') != -1: 
        target3 = list(tmp_df['target3'].unique())
        target3.sort()

    # 2. new_target 컬럼 생성(sido 컬럼이 우선적으로 나오도록 target값 변경)
    if file_nm.find('dis') == -1: 
        if sido == target1:
            tmp_df['new_target'] = tmp_df['target1'] + '_' + tmp_df['target2']
        elif sido == target2:
            tmp_df['new_target'] = tmp_df['target2'] + '_' + tmp_df['target1']
    else:
        if sido == target1:
            tmp_df['new_target'] = tmp_df['target1'] + '_' + tmp_df['target2'] + '_' + tmp_df['target3']
        elif sido == target3:
            tmp_df['new_target'] = tmp_df['target3'] + '_' + tmp_df['target1'] + '_' + tmp_df['target2']
            
#     tmp_df = tmp_df[['new_target','MSE','BEST_MDL']]
    tmp_df['file_nm'] = file_nm.split('/')[1].split('.')[0]
    tmp_df = tmp_df[['file_nm','new_target','MSE','BEST_MDL']]
    tmp_df = tmp_df.sort_values(by = ['new_target'])
    tmp_df.to_csv('mody_month_result/month_' + file_nm.split('/')[1].split('.')[0] + '.csv', index=False, encoding = 'utf-8')

---------

# (경우2) target명 통일시키고 연 MSE값 구하기

In [ ]:
## target명 통일시키고 연 MSE값 구하기
file_list = glob.glob('result/*.csv')
file_list.sort()

for file_nm in file_list:
    
    # 연 MSE값 구하기
    df = pd.read_csv(file_nm, encoding = 'utf-8')
    df['stand_time'] = df['stand_time'].astype('str')
    df['YEAR'] = df['stand_time'].map(lambda x : x[0:4])

    tmp = df.groupby(['target','YEAR']).sum().reset_index()
    tmp['YR_DIFF'] = tmp['PREDICT'] - tmp['SEP_CNT']
    tmp['YR_MSE'] = tmp['YR_DIFF'] * tmp['YR_DIFF']
    tmp_df = tmp[['YEAR','target','PREDICT','SEP_CNT','YR_DIFF','YR_MSE']]  # PREDICT/SEP_CNT/YR_DIFF/YR_MSE : 연 값    
    
    ## < target 컬럼명 조정 > 
    if file_nm.find('dis') == -1:
        tmp_df['target1'] = tmp_df['target'].map(lambda x : x.split('_')[0])
        tmp_df['target2'] = tmp_df['target'].map(lambda x : x.split('_')[1])
    else:
        tmp_df['target1'] = tmp_df['target'].map(lambda x : x.split('_')[0])
        tmp_df['target2'] = tmp_df['target'].map(lambda x : x.split('_')[1])        
        tmp_df['target3'] = tmp_df['target'].map(lambda x : x.split('_')[2])
        
    # 1-1. 기본 data의 sido 컬럼 호출
    data_list = glob.glob('data/*.csv')
    sido = []
    for data_nm in data_list:
        tmp = pd.read_csv(data_nm, encoding = 'cp949')
        sido.append(list(tmp['SIDO'].unique()))

    sido = list(pd.Series(list(itertools.chain.from_iterable(sido))).unique())
    sido.sort()
    
    # 1-2. result 데이터의 sido 컬럼 호출
    target1 = list(tmp_df['target1'].unique())
    target1.sort()
    target2 = list(tmp_df['target2'].unique())
    target2.sort()
    if file_nm.find('dis') != -1: 
        target3 = list(tmp_df['target3'].unique())
        target3.sort()

    # 2. new_target 컬럼 생성(sido 컬럼이 우선적으로 나오도록 target값 변경)
    if file_nm.find('dis') == -1: 
        if sido == target1:
            tmp_df['new_target'] = tmp_df['target1'] + '_' + tmp_df['target2']
        elif sido == target2:
            tmp_df['new_target'] = tmp_df['target2'] + '_' + tmp_df['target1']
    else:
        if sido == target1:
            tmp_df['new_target'] = tmp_df['target1'] + '_' + tmp_df['target2'] + '_' + tmp_df['target3']
        elif sido == target3:
            tmp_df['new_target'] = tmp_df['target3'] + '_' + tmp_df['target1'] + '_' + tmp_df['target2']
            
    tmp_df['file_nm'] = file_nm.split('/')[1].split('.')[0]
    tmp_df = tmp_df[['file_nm','new_target','PREDICT','SEP_CNT','YR_DIFF','YR_MSE']]
    tmp_df = tmp_df.sort_values(by = ['new_target'])
    tmp_df.to_csv('mody_year_result/year_' + file_nm.split('/')[1].split('.')[0] + '.csv', index=False, encoding = 'utf-8')

-------------

# 각 모형별 최종 모형(월/분기/반기/연) 선택

In [1]:
import pandas as pd
import glob
import itertools
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
file_list = glob.glob('mody_year_result/*.csv')
file_list.sort()

for file_nm in file_list:
    
    # (월/분기/반기/연) 결과 병합
    df = pd.read_csv(file_nm, encoding = 'utf-8')
    division_nm = file_nm.split('/')[1].split('.')[0].split('_')[-1]
    
    df = df[['file_nm','new_target','YR_MSE']]
    df = df.rename(columns = {'file_nm':(division_nm + '_file'),'new_target':(division_nm + '_tgt'),'YR_MSE':(division_nm + '_yr_mse')})    
    
    if file_nm.find('_age') != -1:
        if file_nm == 'mody_year_result/year_result_merge_age.csv':
            df_age = df
        else:
            df_age = pd.concat([df_age,df],axis = 1)
    elif file_nm.find('_cls') != -1:
        if file_nm == 'mody_year_result/year_result_merge_cls.csv':
            df_cls = df
        else:
            df_cls = pd.concat([df_cls,df],axis = 1)
    elif file_nm.find('_dis') != -1:
        if file_nm == 'mody_year_result/year_result_merge_dis.csv':
            df_dis = df
        else:
            df_dis = pd.concat([df_dis,df],axis = 1)
    else:
        print('?')

In [3]:
# for divi in ['age','cls','dis']:
#     print('---------------------------------------------------------------------------')
#     print(list(pd.Series(list(globals()['df_' + divi][divi + '_tgt'] == globals()['df_' + divi]['half_tgt'])).unique()))
#     print(list(pd.Series(list(globals()['df_' + divi][divi + '_tgt'] == globals()['df_' + divi]['month_tgt'])).unique()))
#     print(list(pd.Series(list(globals()['df_' + divi][divi + '_tgt'] == globals()['df_' + divi]['quarter_tgt'])).unique()))
#     print('---------------------------------------------------------------------------')

In [4]:
for divi in ['age','cls','dis']:
    globals()['df_' + divi] = globals()['df_' + divi][[divi + '_tgt',divi + '_yr_mse','month_yr_mse','quarter_yr_mse','half_yr_mse']]
    globals()['df_' + divi]['MIN_MSE'] = np.NaN
    globals()['df_' + divi]['OPT_MDL'] = ''
    for i in range(len(globals()['df_' + divi])):
        globals()['df_' + divi]['MIN_MSE'][i] = globals()['df_' + divi][[divi + '_yr_mse','month_yr_mse','quarter_yr_mse','half_yr_mse']].iloc[i].min()
        if globals()['df_' + divi]['MIN_MSE'][i] == globals()['df_' + divi][divi + '_yr_mse'][i]:
            globals()['df_' + divi]['OPT_MDL'][i] = divi + '_yr_mse'
        elif globals()['df_' + divi]['MIN_MSE'][i] == globals()['df_' + divi]['month_yr_mse'][i]:
            globals()['df_' + divi]['OPT_MDL'][i] = 'month_yr_mse'
        elif globals()['df_' + divi]['MIN_MSE'][i] == globals()['df_' + divi]['quarter_yr_mse'][i]:
            globals()['df_' + divi]['OPT_MDL'][i] = 'quarter_yr_mse'
        elif globals()['df_' + divi]['MIN_MSE'][i] == globals()['df_' + divi]['half_yr_mse'][i]:
            globals()['df_' + divi]['OPT_MDL'][i] = 'half_yr_mse'
    globals()['df_' + divi][divi + '_yr_mse'] = round(globals()['df_' + divi][divi + '_yr_mse'],2)
    globals()['df_' + divi]['month_yr_mse'] = round(globals()['df_' + divi]['month_yr_mse'],2)
    globals()['df_' + divi]['quarter_yr_mse'] = round(globals()['df_' + divi]['quarter_yr_mse'],2)
    globals()['df_' + divi]['half_yr_mse'] = round(globals()['df_' + divi]['half_yr_mse'],2)
    globals()['df_' + divi]['MIN_MSE'] = round(globals()['df_' + divi]['MIN_MSE'],2)
            
df_age.to_csv('fin_result/df_age.csv', index=False, encoding = 'utf-8')
df_cls.to_csv('fin_result/df_cls.csv', index=False, encoding = 'utf-8')
df_dis.to_csv('fin_result/df_dis.csv', index=False, encoding = 'utf-8')

In [5]:
df_age['OPT_MDL'].value_counts()

age_yr_mse        171
half_yr_mse        48
quarter_yr_mse     30
month_yr_mse        6
Name: OPT_MDL, dtype: int64

In [6]:
df_cls['OPT_MDL'].value_counts()

cls_yr_mse        41
half_yr_mse        8
quarter_yr_mse     2
Name: OPT_MDL, dtype: int64

In [7]:
df_dis['OPT_MDL'].value_counts()

dis_yr_mse        143
half_yr_mse        15
quarter_yr_mse     10
month_yr_mse        2
Name: OPT_MDL, dtype: int64

-------------